In [1]:
#installation de Chromium
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb -O /tmp/chrome.deb
!sudo apt-get update
!sudo -E apt-get install -y /tmp/chrome.deb
!pip install chromedriver-autoinstaller selenium

import chromedriver_autoinstaller
path_to_web_driver = chromedriver_autoinstaller.install()

--2025-12-14 14:17:03--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb


Resolving dl.google.com (dl.google.com)... 216.58.213.78, 2a00:1450:4007:806::200e
Connecting to dl.google.com (dl.google.com)|216.58.213.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117886796 (112M) [application/x-debian-package]
Saving to: ‘/tmp/chrome.deb’

/tmp/chrome.deb     100%[===================>] 112.42M  82.5MB/s    in 1.4s    

2025-12-14 14:17:05 (82.5 MB/s) - ‘/tmp/chrome.deb’ saved [117886796/117886796]

Hit:1 http://archive.ubuntu.com/ubuntu noble InRelease
Get:2 http://archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]        
Get:3 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]      
Get:4 http://archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]      
Hit:5 https://ppa.launchpadcontent.net/git-core/ppa/ubuntu noble InRelease     
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ubuntugis-unstable/ubuntu noble InRelease
Get:7 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:8 h

In [100]:
#import de modules et initialisation du webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('--verbose')

In [101]:
#lancement du navigateur Chrome
service = Service(executable_path=path_to_web_driver)

browser = webdriver.Chrome(
    service=service,
    options=chrome_options
)

In [102]:
#chargement de la page airparif
browser.get('https://www.airparif.fr/surveiller-la-pollution/bilans-et-cartes-annuels-de-pollution')

In [103]:
#choix de l'année
année="2023"
from selenium.webdriver.support.ui import Select
wait = WebDriverWait(browser, 10)
element_menu = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="date-filter"]')))
menu_deroulant = Select(element_menu)
menu_deroulant.select_by_visible_text(année)

time.sleep(1)

In [104]:
#localisation de la barre des éléments :
from selenium.common.exceptions import StaleElementReferenceException

l=["Palaiseau","Paris 11","Paris 13", "Melun", "Versailles"]

def fonction_taux_de_particule(l):
    search = browser.find_element("xpath",'//*[@id="carte-annuelle-panel-info"]//input')
    valeur=[{"Commune" : e ,"pm10":None , "pm25":None , "no2":None} for e in l]
    liste_polluant=["pm10","pm25","no2"]
    for j in range(len(liste_polluant)):
        #Remarque:initialement on se situe sur le pm10 par défaut
        polluant=liste_polluant[j]
        if j <=1:
            polluant_suivant=liste_polluant[j+1]
        else:
            polluant_suivant=None
        for i in range(len(l)):
            #1/la barre de recherche
            search.clear()
            search.send_keys(l[i])  
            #2/la barre de suggestion
            wait = WebDriverWait(browser, 10, ignored_exceptions=(StaleElementReferenceException,))
            suggestion_a_cliquer= wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"pac-item")))
            print(suggestion_a_cliquer.text)
            suggestion_a_cliquer.click()
            time.sleep(0.1)
            #3/le taux de particules dans l'air
            taux_de_particules=wait.until(EC.visibility_of_element_located((By.XPATH,'//*[@id="current-params"]/strong')))
            valeur[i][polluant]=taux_de_particules.text
        #chargement page nouveau polluant
        print(f"polluant: {polluant} done")
        if polluant_suivant !=None:
            bouton_polluant = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[data-pid={polluant_suivant}]")))
            browser.execute_script("arguments[0].click();", bouton_polluant)
            print(f"Clic forcé sur le {polluant_suivant} effectué via JavaScript.")
            time.sleep(5)
    df=pd.DataFrame(valeur)    
    return(df)

df=fonction_taux_de_particule(l)
print(df)
        

PalaiseauFrance
Paris 11Paris, France
Paris 13Paris, France
MelunFrance
VersaillesFrance
polluant: pm10 done
Clic forcé sur le pm25 effectué via JavaScript.
PalaiseauFrance
Paris 11Paris, France
Paris 13Paris, France
MelunFrance
VersaillesFrance
polluant: pm25 done
Clic forcé sur le no2 effectué via JavaScript.
PalaiseauFrance
Paris 11Paris, France
Paris 13Paris, France
MelunFrance
VersaillesFrance
polluant: no2 done
      Commune pm10 pm25 no2
0   Palaiseau   14    8  15
1    Paris 11   21   11  29
2    Paris 13   20   11  29
3       Melun   18    9  20
4  Versailles   16    9  21


In [89]:
polluant_suivant="pm25"
bouton_pm25 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"a[data-pid={polluant_suivant}]")))
# 2. On clique dessus
browser.execute_script("arguments[0].click();", bouton_pm25)
print("Clic forcé sur PM2.5 effectué via JavaScript.")

Clic forcé sur PM2.5 effectué via JavaScript.
